In [3]:
# always import
import sys
from time import time

# numpy & scipy
import numpy as np
import scipy

# sklearn
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import pairwise_distances_argmin, pairwise_distances
from sklearn import neighbors

# Hungarian algorithm
from munkres import Munkres
from scipy.optimize import linear_sum_assignment

# visuals
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn.manifold import Isomap, TSNE

# maybe
from numba import jit

import scipy.sparse as sparse


In [4]:
# load MNIST data and normalization
from sklearn.datasets import fetch_openml
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, data_home='mnist/')
y = np.asarray(list(map(int, y)))
X = np.asarray(X.astype(float))
X = scale(X)
n_digits = len(np.unique(y))

y_bk = y.copy()

In [5]:
def Kmeans(X,K,max_iter=300,t=0.001, Cinit = 'random'):
    
    num_iter = 0;
    z = np.zeros(X.shape[0], dtype=int)
    J=0
    Jprev = 0
    
    if(Cinit == 'random'):
        C = X[np.random.randint(X.shape[0], size=K)]  
    elif(Cinit == 'PCAcomponents'):
        pca  = PCA()
        pca.fit(X)
        C = pca.components_[0:K,:]
        print("C.shape:", C.shape)
    
    while(num_iter<max_iter):
        num_iter +=1
        C_next = np.zeros(C.shape)
        C_next_cnt = np.zeros(C.shape[0])
        for i in range(X.shape[0]):
            temp = np.array([X[i]]*C.shape[0])
            temp = temp-C
            temp = np.multiply(temp,temp)
            s = np.sum(temp, axis=1)
            z[i] = int(np.argmin(s))
            C_next[z[i]] += X[i]
            C_next_cnt[z[i]] +=1
        
        C_next = C_next/np.array([C_next_cnt]).T 
        C = C_next.copy()
        
        J=0
        for i in range(X.shape[0]):
            x = X[i]
            c = C[z[i]]
            J += np.dot(x-c,x-c)
        
        #print("Iteration "+ str(num_iter) +":" + "J=" + str(J))
        #print("Jprev = ", Jprev)
        if(abs(Jprev - J)< t):
            break
        Jprev = J
    
    print("Final J=", J)
    return z,C,J


In [17]:
#2a-i
z,Co,J = Kmeans(X,K=10,max_iter=300,t=0.001, Cinit = 'PCAcomponents')
z1 = z.copy()

print("Objective = ", J)

m1 = metrics.homogeneity_score(y,z)
print("Homogenity Score =", m1)

m2 = metrics.completeness_score(y,z)
print("Completeness Score =", m2)

m3 = metrics.v_measure_score(y,z)
print("V measure Score =", m3)

m4 = metrics.adjusted_rand_score(y,z)
print("Adjusted rand Score =", m4)

m5 = metrics.adjusted_mutual_info_score(y,z)
print("Adjusted Mutual Info =", m5)

C.shape: (10, 784)
Final J= 42569872.58080556
Objective =  42569872.58080556
Homogenity Score = 0.4198052752753
Completeness Score = 0.4417562572279014
V measure Score = 0.43050113119913763
Adjusted rand Score = 0.3202170656393889
Adjusted Mutual Info = 0.4196593043615676


C:\Users\vinee\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


In [6]:
#2a-ii
def Kmeans_multipleTries(X,K, num_tries):
    bestJ = float('inf')
    bestz = np.zeros(X.shape[0], dtype=int)
    bestCo = np.zeros((X.shape[0],K))
    
    for trial in range(num_tries):
        z,Co,finalJ = Kmeans(X,K,max_iter=300,t=0.001, Cinit = 'random')
        if(finalJ < bestJ):
            bestJ = finalJ
            bestz = z.copy()
            bestCo = Co.copy()
            
    return bestz, bestCo, bestJ

In [77]:

    

#2a-ii
pca  = PCA(n_components=30)
pca.fit(X)  
Xdash = pca.transform(X)

bestz, bestCo, bestJ = Kmeans_multipleTries(Xdash,K=10, num_tries=10)

       
print("Best objective = ", bestJ)  
z2 = bestz.copy()

m1 = metrics.homogeneity_score(y,bestz)    
print("Homogenity Score =", m1)

m2 = metrics.completeness_score(y,bestz)
print("Completeness Score =", m2)

m3 = metrics.v_measure_score(y,bestz)
print("V measure Score =", m3)

m4 = metrics.adjusted_rand_score(y,bestz)
print("Adjusted rand Score =", m4)

m5 = metrics.adjusted_mutual_info_score(y,bestz)
print("Adjusted Mutual Info =", m5)   

Final J= 15075079.049295807
Final J= 15012221.894483838
Final J= 15012217.349797744
Final J= 15075529.50290776
Final J= 15078750.711568555
Final J= 15012690.732292935
Final J= 15012230.465892917
Final J= 15012214.326240761
Final J= 15078895.571706085
Final J= 15077719.141220408
Best objective =  15012214.326240761
Homogenity Score = 0.4220445033066962
Completeness Score = 0.44380726213297633
V measure Score = 0.4326523845931157
Adjusted rand Score = 0.32189754467354037
Adjusted Mutual Info = 0.4218990968210131


C:\Users\vinee\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:746: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


In [7]:
# TODO: Hungarian algorithm

#2b Hungarian
def hungarian(z,y,K):
    """
    z = label o/p from clustering
    k = number of labels
    
    returns:
        col_ind - mapping
        zdash - mapped z's
    """
    W = np.zeros((K,K))
    n = z.shape[0]
    for i in range(K): #z
        zi = np.array([k for k, x in enumerate(z) if x == i])
        for j in range(K): #y
            yj = np.array([l for l, x in enumerate(y) if x == j])
            common = list(set(zi).intersection(yj))
            W[i,j] = 1- len(common)/n
            
    minimum = W.min(axis=1)
    minimum = np.array([minimum]*K).T
    W = W-minimum
    row_ind, col_ind = scipy.optimize.linear_sum_assignment(W)
    
    #compute zdash
    zdash = np.zeros(z.shape, dtype=int)
    for i in range(z.shape[0]):  
        zdash[i] = col_ind[z[i]]
                
    return col_ind, zdash



In [8]:
def accuracy(prediction,y):
    correct = 0
    for j in range(len(y)):
        if(prediction[j]==y[j]):
            correct +=1
    prec_test = correct/len(prediction) 
    return prec_test

In [70]:
#2b
#Using z1 from 2a-i
col_ind, zdash = hungarian(z1,y,K)
conf_matrix = confusion_matrix(y,zdash)
acc = accuracy(zdash,y)

print("Mapping:", col_ind)
print("Confusion Matrix:")
print(conf_matrix)
print("Accuracy = ", acc)

Mapping: [0 9 2 6 1 3 8 7 5 4]
Confusion Matrix:
[[3819   36  113 1402   12  520  355    6  630   10]
 [   0 7644   13   26    8  154   16    5   10    1]
 [  37  823 2387  717  191   69  853   32 1840   41]
 [   9  573  746 4087  204   99   87   97 1134  105]
 [  55  535   73    5 3969  934  128  754   28  343]
 [  31  465  247 2161  311 2669  102   88  176   63]
 [ 291  566  416  113   28  119 5323    1   14    5]
 [  20  516   16    9 1581  137    3 4082   13  916]
 [  44 1175  206 2627  358 2031   32  189   90   73]
 [  44  332   23  124 3485  124    5 2374   16  431]]
Accuracy =  0.49287142857142857


In [84]:
#Using z2 from 2a-ii
col_ind, zdash = hungarian(z2,y,K)
conf_matrix = confusion_matrix(y,zdash)
acc = accuracy(zdash,y)


print("Mapping:", col_ind)
print("Confusion Matrix:")
print(conf_matrix)
print("Accuracy = ", acc)

Mapping: [8 5 0 7 2 4 9 6 1 3]
Confusion Matrix:
[[3970   35   83  930   22  662  767    7  414   13]
 [   0 7621   13   29    5  173   16    7   12    1]
 [  42  842 2583  773  163   94  677   44 1717   55]
 [  10  679  220 4443  185  156   97   92 1138  121]
 [  52  543   49    3 3928  906  156  688   24  475]
 [  36  533  128 2058  328 2794  129   71  157   79]
 [ 239  505  773   54   37  133 5119    1    9    6]
 [  20  544    7   13 1585  117    3 4080   18  906]
 [  48 1341   96 2221  399 2300   52  184   78  106]
 [  41  359   15  105 3490  120    6 2291   14  517]]
Accuracy =  0.5019


In [20]:
# TODO: Spectral clustering
def SpectralClustering(X,K=10, n_neighbours=500, k=20):
    """
    X: The input data 
    K: K for Kmeans (num_clusters)
    n_neighbours:  for teh nearest neighbour
    k: Choose k smallest eigen values
    
    Returns:
    best_z, best_Co, best_obj
    """

    #Compute H from nearest neighbour graph
    nn = neighbors.NearestNeighbors(n_neighbors=500,algorithm='kd_tree',metric='euclidean')
    H = nn.fit(X).kneighbors_graph(mode = 'distance') #H is parse

    #Compute sigma
    sigma = H.sum()/H.count_nonzero() #10.815
    print("sigma=", sigma)
    E = H.power(2)
    E = E/(sigma*sigma) #sparse

    #Compute E 
    E.data = np.exp(-(0.02*E.data))

    #Normalise E  
    #S = sparse.csr_matrix(1/E.sum(axis=1))
    #E = E.multiply(S)

    np.divide(E.data, E.sum(), out=E.data)
    E.setdiag(1.0)

    #Make E symmetric
    E  = (E+E.transpose())/2

    #Compute D
    e = np.array(E.sum(axis=1)).reshape(1,-1)[0]
    e2 = np.power(e,-0.5)
    D = sparse.diags(e)
    D2 = sparse.diags(e2)

    #Compute L
    n = X.shape[0]
    L = sparse.identity(n) - D2@E@D2
    e_val, e_vec = sparse.linalg.eigs(L, k, which='SM')
    print("eigen values:", e_val)

    e_vec = np.delete(e_vec,0,1)
    #e_vec = np.delete(e_vec,0,1)

    #Calculate U
    U = e_vec/np.linalg.norm(e_vec, axis=1).reshape(-1,1)

    #Run k-means 10 times and choose the best
    best_obj = float('inf')
    best_z = np.zeros(X.shape[0], dtype=int)
    best_Co = np.zeros((X.shape[0],10))

    for trial in range(10):
        kmeans = KMeans(init='k-means++', n_clusters=K, n_init=10).fit(np.real(U))
        print("K Means Trial" + str(trial) + ": Objective=" + str(kmeans.inertia_) + ", max_iter=" + str(kmeans.max_iter))
        if(kmeans.inertia_ < best_obj):
            best_obj = kmeans.inertia_
            best_z = kmeans.labels_.copy()
            best_Co = kmeans.cluster_centers_.copy()
            
    return best_z, best_Co, best_obj,U

In [21]:
#2c

pca  = PCA(n_components=30)
pca.fit(X)  
X1dash = pca.transform(X)

best_z, best_Co, best_obj,U = SpectralClustering(X1dash,K=10, n_neighbours=500, k=20)



sigma= 10.827380619841561


C:\Users\vinee\Anaconda3\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


eigen values: [-6.72067704e-19+0.j  4.39544059e-07+0.j  5.01670092e-07+0.j
  7.32078723e-07+0.j  7.82265832e-07+0.j  8.24910419e-07+0.j
  9.34366477e-07+0.j  1.09760248e-06+0.j  1.19642650e-06+0.j
  1.29473680e-06+0.j  1.37233893e-06+0.j  1.47307052e-06+0.j
  1.54284439e-06+0.j  1.62902265e-06+0.j  1.66207008e-06+0.j
  1.69542867e-06+0.j  1.86734265e-06+0.j  2.07265437e-06+0.j
  2.28667818e-06+0.j  2.37483503e-06+0.j]
K Means Trial0: Objective=33680.35868110175, max_iter=300
K Means Trial1: Objective=33676.75185949395, max_iter=300
K Means Trial2: Objective=33573.60970068055, max_iter=300
K Means Trial3: Objective=33476.69318049917, max_iter=300
K Means Trial4: Objective=33459.82939039365, max_iter=300
K Means Trial5: Objective=33646.40854171446, max_iter=300
K Means Trial6: Objective=33476.707363003356, max_iter=300
K Means Trial7: Objective=33573.61189085531, max_iter=300
K Means Trial8: Objective=33476.69318049917, max_iter=300
K Means Trial9: Objective=33573.62493210252, max_iter=3

In [22]:
mapping1, zdash1 = hungarian(best_z,y_bk,10)
conf_matrix1 = confusion_matrix(y_bk,zdash1)
acc = accuracy(zdash1,y_bk)

print("Mapping:", mapping1)
print("Confusion Matrix:")
print(conf_matrix1)
print("Accuracy = ", acc)

Mapping: [4 2 3 6 9 5 0 1 8 7]
Confusion Matrix:
[[4427    1    9   34    5 2336   43    3   40    5]
 [   1 3524   30   14    9 4254   21    2   14    8]
 [  83   41 5769  258   17  402   40   68  286   26]
 [  20  110  102 5136   21 1030   17   89  525   91]
 [   3   64   56   20 3248  552   35   16    4 2826]
 [  60   38   22 1783   45 3201  140    3  857  164]
 [  55   21   25   28    5 1417 5295    1   24    5]
 [   3   93   32   23  368 1379    0 4864    8  523]
 [  42   73   36  915   77  674   24   13 4856  115]
 [  23   51   27   84 2774  678    1  126   63 3131]]
Accuracy =  0.6207285714285714


In [18]:
#k-NN
class kNN(object):
    
    def __init__(self, k, X, y):
        self.k = k
        self.X = X
        self.y = y
        
    def distance(self,a):
        """
        a is 1xm, ie one data
        distance returns X.shape[0] x 1 array with disatnces to each of the datapoints in X
        """
        temp = np.array([a]*self.X.shape[0])
        temp = temp-self.X
        temp = np.multiply(temp,temp)
        dist = np.sqrt(np.sum(temp, axis=1))
        return dist

        
    def predict(self,test):
        """
        test is nxm
        """
        pred = np.zeros(test.shape[0])
        for i in range(test.shape[0]):
            row = test[i]
            dist = self.distance(row)
            #Append dist and y
            dist_y = np.column_stack((dist,self.y))
            dist_y = dist_y[np.argsort(dist_y[:,0])]
            y_n = np.array(dist_y[0:self.k,1], dtype='int')
            counts = np.bincount(y_n)
            pred[i] = np.argmax(counts)
            
        return pred
            
"""
#Test kNN
X =  np.array([[0,0],[1,1],[2,2],[3,3],[4,4],[5,5],[6,6]])
y = np.array([1,1,1,0,0,0,1])

test = np.array([[-1,-1],[3.5,3.5]])

my_kNN = kNN(4,X,y)      
pred = my_kNN.predict(test)
"""

'\n#Test kNN\nX =  np.array([[0,0],[1,1],[2,2],[3,3],[4,4],[5,5],[6,6]])\ny = np.array([1,1,1,0,0,0,1])\n\ntest = np.array([[-1,-1],[3.5,3.5]])\n\nmy_kNN = kNN(4,X,y)      \npred = my_kNN.predict(test)\n'

In [14]:
#2d
#Do PCA to get Xdash
pca  = PCA(n_components=30)
pca.fit(X)  
Xdash = pca.transform(X)

In [13]:
#2d - Random selection  + kNN

#Get 100 points from Xdash, random selection
ind = np.random.randint(Xdash.shape[0], size=100)
trainX_knn = Xdash[ind]
trainy_knn = y[ind]

testX_knn = np.delete(Xdash,ind, axis=0)
testy_knn_true = np.delete(y, ind)

#Do kNN on those 100 points

#k=1
mykNN = kNN(1,trainX_knn, trainy_knn)
#Test accuracy on the remaining 69900 points
pred = mykNN.predict(testX_knn)
#accuracy
acc = accuracy(pred, testy_knn_true)
print("Random Selection + kNN, k=1: Accuracy=", acc)

#k=3
mykNN = kNN(3,trainX_knn, trainy_knn)
#Test accuracy on the remaining 69900 points
pred = mykNN.predict(testX_knn)
#accuracy
acc = accuracy(pred, testy_knn_true)
print("Random Selection + kNN, k=3: Accuracy=", acc)
     
            
#k=5
mykNN = kNN(5,trainX_knn, trainy_knn)
#Test accuracy on the remaining 69900 points
pred = mykNN.predict(testX_knn)
#accuracy
acc = accuracy(pred, testy_knn_true)
print("Random Selection + kNN, k=5: Accuracy=", acc) 

Random Selection + kNN, k=1: Accuracy= 0.720901287553648
Random Selection + kNN, k=3: Accuracy= 0.6713590844062947
Random Selection + kNN, k=5: Accuracy= 0.6413876967095852


In [36]:
#########################################################################
#2d Kmeans+kNN
########################################################################
#Get 100 points using Kmeans      
z,Co,J = Kmeans_multipleTries(Xdash, K=100, num_tries=10)

#Find the 100 points, closest to each of the centroids
ind = np.zeros(100, dtype=int)
for i in range(Co.shape[0]):
    c = Co[i]
    temp = np.array([c]*Xdash.shape[0])
    temp = temp-Xdash
    temp = np.multiply(temp,temp)
    dist = np.sqrt(np.sum(temp, axis=1))
    ind[i] = np.argmin(dist)
    
trainX_knn = Xdash[ind]
trainy_knn = y[ind]

testX_knn = np.delete(Xdash,ind, axis=0)
testy_knn_true = np.delete(y, ind)

    
#Do kNN on those 100 points
#k=1
mykNN = kNN(1,trainX_knn, trainy_knn)
#Test accuracy on the remaining 69900 points
pred = mykNN.predict(testX_knn)
#accuracy
acc = accuracy(pred, testy_knn_true)
print("Kmeans(K=100) + kNN(k=1): Accuracy=", acc) 

#k=3
mykNN = kNN(3,trainX_knn, trainy_knn)
#Test accuracy on the remaining 69900 points
pred = mykNN.predict(testX_knn)
#accuracy
acc = accuracy(pred, testy_knn_true)
print("Kmeans(K=100) + kNN(k=3): Accuracy=", acc)

#k=1
mykNN = kNN(5,trainX_knn, trainy_knn)
#Test accuracy on the remaining 69900 points
pred = mykNN.predict(testX_knn)
#accuracy
acc = accuracy(pred, testy_knn_true)
print("Kmeans(K=100) + kNN(k=5): Accuracy=", acc)

Final J= 7661007.948906397
Final J= 7680293.106257867
Final J= 7684769.557073417


C:\Users\vinee\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in true_divide


Final J= 22686659.30868819
Final J= 7659409.395387609
Final J= 7663865.343802184
Final J= 22686659.30868819
Final J= 7683019.598066069
Final J= 7653528.55860144
Final J= 7663081.353790393
Kmeans(K=100) + kNN(k=1): Accuracy= 0.8183834048640916
Kmeans(K=100) + kNN(k=3): Accuracy= 0.771587982832618
Kmeans(K=100) + kNN(k=5): Accuracy= 0.7500715307582261


In [23]:
############################################################
# 2d: Clustering + KNN
############################################################

#Get 100 points using Spectral Clustering
z, Co, J,U = SpectralClustering(Xdash,K=100, n_neighbours=500, k=20)




sigma= 10.825424107948928


C:\Users\vinee\Anaconda3\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


eigen values: [3.74282684e-19+0.j 4.36824110e-07+0.j 4.99618676e-07+0.j
 7.28040650e-07+0.j 7.78866138e-07+0.j 8.22790696e-07+0.j
 8.61423672e-07+0.j 9.23190650e-07+0.j 9.34264143e-07+0.j
 1.09787716e-06+0.j 1.29459162e-06+0.j 1.47518332e-06+0.j
 1.53708515e-06+0.j 1.62093685e-06+0.j 1.65360044e-06+0.j
 1.69401986e-06+0.j 1.86826141e-06+0.j 2.07302349e-06+0.j
 2.28900422e-06+0.j 2.36696271e-06+0.j]
K Means Trial0: Objective=5178.729735290092, max_iter=300
K Means Trial1: Objective=5169.577888981079, max_iter=300
K Means Trial2: Objective=5177.426316396795, max_iter=300
K Means Trial3: Objective=5174.3791031536275, max_iter=300
K Means Trial4: Objective=5184.910364939011, max_iter=300
K Means Trial5: Objective=5182.939286265331, max_iter=300
K Means Trial6: Objective=5178.307500452593, max_iter=300
K Means Trial7: Objective=5183.132453868287, max_iter=300
K Means Trial8: Objective=5173.127132371928, max_iter=300
K Means Trial9: Objective=5180.227828797855, max_iter=300


In [24]:
Co.shape

(100, 19)

In [25]:

#Find the 100 points, closest to each of the centroids
ind = np.zeros(100, dtype=int)
for i in range(Co.shape[0]):
    c = Co[i]
    temp = np.array([c]*U.shape[0])
    temp = temp-U
    temp = np.multiply(temp,temp)
    dist = np.sqrt(np.sum(temp, axis=1))
    ind[i] = np.argmin(dist)
    
trainX_knn = U[ind]
trainy_knn = y[ind]

testX_knn = np.delete(U,ind, axis=0)
testy_knn_true = np.delete(y, ind)

#Do kNN on those 100 points
#k=1
mykNN = kNN(1,trainX_knn, trainy_knn)
#Test accuracy on the remaining 69900 points
pred = mykNN.predict(testX_knn)
#accuracy
acc = accuracy(pred, testy_knn_true)
print("Spectral + kNN(k=1): Accuracy=", acc) 

#k=3
mykNN = kNN(3,trainX_knn, trainy_knn)
#Test accuracy on the remaining 69900 points
pred = mykNN.predict(testX_knn)
#accuracy
acc = accuracy(pred, testy_knn_true)
print("Spectral + kNN(k=3): Accuracy=", acc)

#k=1
mykNN = kNN(5,trainX_knn, trainy_knn)
#Test accuracy on the remaining 69900 points
pred = mykNN.predict(testX_knn)
#accuracy
acc = accuracy(pred, testy_knn_true)
print("Spectral + kNN(k=5): Accuracy=", acc)

C:\Users\vinee\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: ComplexWarning: Casting complex values to real discards the imaginary part


Spectral + kNN(k=1): Accuracy= 0.8249785407725322
Spectral + kNN(k=3): Accuracy= 0.8171530758226038
Spectral + kNN(k=5): Accuracy= 0.8005436337625179


In [66]:
ind

array([22254, 24140,  3381, 31960, 38065, 37576, 61075, 42792, 44353,
        2791,  6697, 31739, 21846, 45523,    26, 39978, 11807, 22068,
       17122, 33537, 66204, 12786, 23741,  9386, 33049, 11798, 32961,
       65175, 13385, 43764, 29218, 27404, 48068,  6787, 38384, 29042,
       19013,  3444, 42650,   217, 40779, 59846, 37773, 48834, 64456,
       54001, 63782, 57238, 64362, 35310, 23433, 52998, 19028, 52322,
       66045, 20725,  3448, 45582, 67209, 45192, 12625, 32991, 14367,
       31352, 13644, 17573, 14513,  3107, 56481,  2760, 14326,  1018,
       54268,  5939, 41401, 47196,  5568, 56467, 14496, 49167, 49206,
       30387,  3441, 35970, 42519, 53236, 39576,   419, 36122, 58302,
       57123,  3623, 60129, 58642, 10500, 41730, 20518, 45859, 38746,
       19012])

In [59]:
Co


array([[ 0.2002279 ,  0.20198487,  0.03255339, ..., -0.1071951 ,
         0.23571432,  0.00560531],
       [-0.16243327, -0.06561729, -0.03930195, ...,  0.03929089,
        -0.16720723, -0.03404421],
       [ 0.30101349,  0.30184853,  0.07057782, ..., -0.29845742,
        -0.05138567, -0.022496  ],
       ...,
       [ 0.37042895, -0.46557326, -0.02571536, ..., -0.00764065,
         0.00717715,  0.00600808],
       [ 0.21114732,  0.44322426,  0.08912704, ...,  0.25383068,
         0.09085669,  0.00703562],
       [-0.0213168 ,  0.10416908,  0.01870799, ..., -0.35174741,
        -0.03818356, -0.03543201]])